In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/job-dataset/unlabelled_development_set.csv
/kaggle/input/job-dataset/no_agument_training_5118.json
/kaggle/input/job-dataset/agument_work_5316.json
/kaggle/input/job-dataset/prompts_complete_test_salary.json
/kaggle/input/job-dataset/work_arrangements_development_set.csv
/kaggle/input/job-dataset/prompts_complete_test_work.json
/kaggle/input/job-dataset/work_arrangements_test_set.csv
/kaggle/input/job-dataset/combined_prompts_complete_6k.json
/kaggle/input/job-dataset/seniority_labelled_test_set.csv
/kaggle/input/job-dataset/agument_all_7998.json
/kaggle/input/job-dataset/agument_training_7800.json
/kaggle/input/job-dataset/seniority_labelled_development_set.csv
/kaggle/input/job-dataset/unlabelled_development_set.json
/kaggle/input/job-dataset/prompts_complete_test_seniority.json
/kaggle/input/job-dataset/salary_labelled_development_set.csv
/kaggle/input/job-dataset/test_1355.json
/kaggle/input/job-dataset/combined_prompts_complete_train_80.json
/kaggle/input/job-dataset

In [2]:
import os
import time
import torch
from datasets import load_dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import (
    get_peft_model,
    PromptTuningConfig,
    LoraConfig,
    PeftModel,
    TaskType
)
import pandas as pd
from torch.cuda import max_memory_allocated, memory_allocated, memory_reserved

2025-06-29 10:50:33.989491: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751194234.011118     119 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751194234.017805     119 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0' # Setting GPU
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

model_path = "Qwen/Qwen2.5-0.5B-Instruct"
UNLABELLED_CSV = "/kaggle/input/job-dataset/unlabelled_development_set.json" # Make sure to change path
OUTPUT_DIR = "/dapt_checkpoints"


In [4]:
# Examine the unlabelled set
input_df = pd.read_json(UNLABELLED_CSV)
input_df

,prompt
0,Plastics Die Operator A fantastic opportunity ...
1,EL1 Annual Report Coordinator We are looking f...
2,Trades Assistants- Expression of Interest CHAM...
3,Enrolled Endorsed Nurses (EEN) - Noosaville We...
4,Part time Café All-rounder in Liverpool A rest...
...,...
790,E-commerce Key Account Manager e-Commerce Key ...
791,Tiktok Content Creator & Video designer (รับด่...
792,BA (Business Analyst) Responsibilitiesเก็บข้อม...
793,ช่างเทคนิค Job Descriptionsดูแลระบบอาคาร เช่น ...


In [5]:
raw = load_dataset("json", data_files=UNLABELLED_CSV)['train']
raw = raw.remove_columns([c for c in raw.column_names if c != "prompt"])
dataset = DatasetDict({
    "train": raw # Use the entire dataset for training
})

In [6]:
tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    trust_remote_code=True,
    use_fast=False
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


In [7]:
def tokenize_function(examples):
    return tokenizer(
        examples["prompt"], 
        padding="max_length", 
        truncation=True,
        max_length=512)

dataset_tokenized = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["prompt"]
)

# Remove empty sequence
dataset_tokenized['train'] = dataset_tokenized['train'].filter(lambda x: len(x['input_ids']) > 1)


In [8]:
# Prepare batches of data for training
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False # Each token is used to predict next token when mlm=False (Used for DAPT)
)


In [9]:
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    trust_remote_code=True
)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [10]:
dapt_dir = './model/dapt1' # Make sure to change path

dapt_args = TrainingArguments(
    output_dir=dapt_dir,
    num_train_epochs=4,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4, # Accumulate gradients over 4 steps before update
    learning_rate=5e-6,
    save_steps=500,
    logging_steps=20,
    report_to=[],
    optim="adamw_torch" # More memory efficient. The default is adamw_hf
)

trainer = Trainer(
    model=model,
    args=dapt_args,
    train_dataset=dataset_tokenized['train'],
    data_collator=data_collator
)

print("\n== Starting DAPT Training ==")
trainer.train()
model.save_pretrained(dapt_dir)
tokenizer.save_pretrained(dapt_dir)
print("\n== DAPT Training Complete ==")




== Starting DAPT Training ==


Step,Training Loss
20,3.177000
40,3.167500
60,3.134400
80,3.014200
100,3.014100
120,2.808200
140,2.674600
160,2.693700
180,2.727800
200,2.783800



== DAPT Training Complete ==


In [11]:
# Aggressive memory clearing 
import gc
import torch
def print_gpu_memory():
    print(f"Allocated: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    print(f"Cached: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")

# Use it before and after memory clearing
print(f'\nBefore clearing:')
print_gpu_memory()

# Clear any cached memory
torch.cuda.empty_cache()
# Force garbage collection
gc.collect()
# Clear any remaining CUDA memory
torch.cuda.synchronize()

# Reset CUDA device
torch.cuda.reset_peak_memory_stats()
torch.cuda.reset_accumulated_memory_stats()

def print_gpu_memory():
    print(f"Allocated: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    print(f"Cached: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")

# ... memory clearing steps ...
print(f'\nAfter clearing:')
print_gpu_memory()


Before clearing:
Allocated: 5.56 GB
Cached: 12.34 GB

After clearing:
Allocated: 5.56 GB
Cached: 6.84 GB


In [12]:
# After DAPT training is complete
print("\n== Starting Prompt Tuning Training ==")

peft_config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    prompt_tuning_init="TEXT",
    prompt_tuning_init_text="Extract key information from the text:",  # Initial English text
    num_virtual_tokens=20,
    tokenizer_name_or_path='Qwen/Qwen2.5-0.5B-Instruct',
    token_dim=model.config.hidden_size,
    num_transformer_submodules=1
)

# Make sure to check where dapt is stored
dapt_model_path = "/kaggle/working/model/dapt1"

# Load fresh model
model = AutoModelForCausalLM.from_pretrained(
    dapt_model_path,
    trust_remote_code=True,
    torch_dtype=torch.float16,
    device_map="auto"
)


# Apply PromptTuning peft to the existing model
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# # Now move the Prompt model to GPU
# model.to("cuda")



== Starting Prompt Tuning Training ==
trainable params: 17,920 || all params: 494,050,688 || trainable%: 0.0036


In [13]:
# Load Training set
train_input_path = "/kaggle/input/job-dataset/agument_work_5316.json"
train_input_df = pd.read_json(train_input_path)
train_input_df

,prompt,complete
0,[TASK: Salary] Extract salary information from...,17500-17500-PHP-MONTHLY
1,[TASK: Salary] Extract salary information from...,16000-16000-PHP-MONTHLY
2,[TASK: Salary] Extract salary information from...,0-0-None-None
3,[TASK: Salary] Extract salary information from...,0-0-None-None
4,[TASK: Salary] Extract salary information from...,0-0-None-None
...,...,...
5311,[TASK: Work Arrangement] Extract work arrangem...,Remote
5312,[TASK: Work Arrangement] Extract work arrangem...,Hybrid
5313,[TASK: Work Arrangement] Extract work arrangem...,Hybrid
5314,[TASK: Work Arrangement] Extract work arrangem...,Hybrid


In [14]:
def load_and_preprocess(data_path):
    dataset = load_dataset("json", data_files=data_path)

    dataset = dataset.filter(lambda x: len(x["prompt"]) > 0 and len(x["complete"]) > 0)

    def tokenize_function(examples):
        # print(examples)
        completions = [str(c) for c in examples["complete"]]

        texts = [p + c + tokenizer.eos_token for p, c in zip(examples["prompt"], completions)]

        tokenized = tokenizer(
            texts,
            max_length=512,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )

        prompt_tokenized = tokenizer(examples["prompt"], add_special_tokens=False)
        prompt_lengths = [len(ids) for ids in prompt_tokenized["input_ids"]]

        labels = tokenized["input_ids"].clone()
        for i, length in enumerate(prompt_lengths):
            if length >= 2048:
                length = 2047
            labels[i][:length] = -100

        tokenized["labels"] = labels
        return tokenized

    return dataset.map(tokenize_function, batched=True)

In [15]:
def formal_train(json_path="salary_47_allin.json", target_model_path="./qwen_prompt_47"):
    dataset = load_and_preprocess(json_path)
    data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

    training_args = TrainingArguments(
        output_dir="./qwen_dapt_prompt_output", # saving training checkpoint
        per_device_train_batch_size=4,
        gradient_accumulation_steps=2,
        num_train_epochs=6,
        learning_rate=8e-4,
        logging_dir="./logs",
        logging_steps=100,
        save_strategy="epoch",
        fp16=True,
        optim="adamw_torch",
        dataloader_num_workers=2,
        report_to="none"
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset["train"],
        data_collator=data_collator,
    )

    print(f"\nTraining Start...")
    start_time = time.time()
    trainer.train()
    print(f"\nTraining Complete，Time: {time.time() - start_time:.2f}s")

    model.save_pretrained(target_model_path)

    # To check memory useage
    # free unused memory before checking
    torch.cuda.empty_cache()
    # Memory usage check (in GB)
    print(f"\nPeak memory usage: {max_memory_allocated() / 1024**3:.2f} GB")



In [16]:
# Make sure to adjust path
saving_final_model_path = './qwen_dapt_prompt_augwork_final_model'
formal_train(json_path=train_input_path, target_model_path=saving_final_model_path)

Generating train split: 0 examples [00:00, ? examples/s]

Filter:   0%|          | 0/5316 [00:00<?, ? examples/s]

Map:   0%|          | 0/5316 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.



Training Start...


Step,Training Loss
100,3.013200
200,2.877800
300,2.858700
400,2.829600
500,2.810800
600,2.803700
700,2.799200
800,2.791100
900,2.778400
1000,2.792600



Training Complete，Time: 6242.40s

Peak memory usage: 12.26 GB
